# <font color='blue'>ALPAR - Governo Digital - Gerador de Dados</font>

In [1]:
# install conector MySQL
# !pip install mysql-connector-python

#install pyDF2
# !pip install PyPDF2

#install timestring
#!pip install timestring


In [2]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import random as rd
import string
import timestring as ts
import datetime as dt
from os.path import expanduser

print("pandas versão", pd.__version__)
print("numpy versão", np.__version__)
print("csv versão", csv.__version__)
print("csv timestring", ts.__version__)

pd.options.display.max_rows = 2000

pandas versão 1.1.5
numpy versão 1.19.4
csv versão 1.0
csv timestring 1.6.2


# <font color='black'>parâmetros gerais</font>

In [3]:
# parâmetros cadastrais
pr = {
    'cidades': (20, 30),           # (min, max) - total de cidades que fará parte da amostra para entidades e conteúdo de formulários
    'entidades': 0.8,              # qtd de entidades - % sobre a qtd de cidades
    'servico': {
        'tot': 200,                # lista total de serviços (de 1 a n)
        'min': 60,                 # número mínimo de serviços que fará parte da simulação
        'max': 100,                # número máximo de serviços que fará parte da simulação
        'prazo': (7000, 60000),    # intervalo de prazo para SLA
        'faixa_amarela': 0.85      # porcentagem da faixa amarela em relação a faixa vermelha
    },
    'categoria_servico': {
        'tot': 100,                # lista total de categoria de serviços (de 1 a n)
        'min': 4,                  # número mínimo de categoria de serviços que fará parte da simulação
        'max': 7                   # número máximo de categoria de serviços que fará parte da simulação
    },
    'grupo_responsavel': {
        'tot': 100,                # lista total de grupos responsáveis (de 1 a n)
        'min': 2,                  # número mínimo de grupo responsável que fará parte da simulação
        'max': 6                   # número máximo de grupo responsável que fará parte da simulação
    },
    'usuario': {
        'tot': 100,                # lista total de usuários (de 1 a n)
        'min': 30,                 # número mínimo de usuários que fará parte da simulação
        'max': 40                  # número máximo de usuários que fará parte da simulação
    },
    'grupo_usuario': {
        'tot': 50,                 # lista total de grupos de usuários (de 1 a n)
        'min': 10,                 # número mínimo de grupos de usuários que fará parte da simulação
        'max': 14                  # número máximo de grupos de usuários que fará parte da simulação
    },
    'departamento': {
        'tot': 100,                # lista total de departamentos de encaminhamento (de 1 a n)
        'min': 10,                 # número mínimo de departamentos de encaminhamento que fará parte da simulação
        'max': 14                  # número máximo de departamentos de encaminhamento que fará parte da simulação
    },
    'motivo_cancelamento': {       
        'tot': 20,                 # lista total de motivos de cancelamento (de 1 a n)
        'min': 5,                  # número mínimo de motivos de cancelamento que fará parte da simulação
        'max': 8                   # número máximo de motivos de cancelamento que fará parte da simulação
    },
    'status_ext': {
        'tot': 100,                # lista total de status externo (de 1 a n)
        'min': 10,                 # número mínimo de status externo que fará parte da simulação 
        'max': 20                  # número máximo de status externo que fará parte da simulação
    },
    'motivo_rating': {
        'tot': 100,                # lista total motivos de rating (de 1 a n)
        'min': 5,                  # número mínimo de motivos de rating que fará parte da simulação 
        'max': 8                   # número máximo de motivos de rating que fará parte da simulação
    },
    'servicos_por_entidade': {
        'min': 10,                      # número mínimo de serviços por entidade
        'max': 20,                      # número máximo de serviços por entidade
        'atributos_form': (6, 10),      # número de atributos de formulário por serviço (mínimo, máximo)
        'atributo_integer': (1, 100),   # intervalo de valores para atributos do tipo INTEGER
        'atributo_string': (1, 5)       # intervalo de valores para atributos do tipo STRING combinado com descrição
    },
    'cidades_por_entidade': (10, 20),   # quantidade de cidades para ser randomizado no conteúdo dos formulários
    'acao': {
        'tot': 100,                # lista total de ações (de 1 a n)
        'min': 10,                 # número mínimo de ações por entidade
        'max': 20,                 # número máximo de ações por entidade
        'prop_notificacao': (7, 3) # proporção média de ações com notificações - (com notificação, sem notificação)
    },
    'categorias_servico_por_entidade': {
        'min': 5,                  # número mínimo de categorias de serviço por entidade
        'max': 7                   # número máximo de categorias de serviço por entidade
    },
    'usuários_por_entidade': {
        'min': 10,                 # número mínimo de usuários por entidade
        'max': 20                  # número máximo de usuários por entidade
    },
    'grupos_usuários_por_entidade': {
        'min': 10,                 # número mínimo de grupos de usuários por entidade
        'max': 20                  # número máximo de grupos de usuários por entidade
    },
    'tasks_por_protocolo': {
        'cancelado': {
            'cod': 3,                              # código numérico para protocolos cancelados
            'min': 2,                              # número mínimo de grupos de usuários por entidade
            'max': 8,                              # número máximo de grupos de usuários por entidade
            'nome': 'Cancelado',                   # nome da situação
            'nome_task': 'Protocolo Cancelado',    # nome da última task da coleção
            'peso_prop': 20                        # peso para proporção na craição aleatória
        },
        'concluido': {
            'cod': 2,                              # código numérico para protocolos concluídos
            'min': 5,                              # número mínimo de grupos de usuários por entidade
            'max': 12,                             # número máximo de grupos de usuários por entidade
            'nome': 'Concluído',                   # nome da situação
            'nome_task': 'Protocolo Concluído',    # nome da última task da coleção
            'peso_prop': 70                        # peso para proporção na craição aleatória
        },
        'andamento': {
            'cod': 1,                              # código numérico para protocolos em andamento
            'min': 2,                              # número mínimo de grupos de usuários por entidade
            'max': 11,                             # número máximo de grupos de usuários por entidade
            'nome': 'Em Andamento',                # nome da situação
            'nome_task': '',                       # nome da última task da coleção
            'peso_prop': 10                        # peso para proporção na craição aleatória
        },
        'tempos': {                                # tempos mínimos e máximos a serem estipulados (em dias)
            'min': 0.5,
            'max': 10
        },
        'gap_entre_tasks': {
            'min': 0.05,
            'max': 1.0
        },
        'gap_atendimento': {
            'min': 0.05,
            'max': 0.1,
            'prop_anda_atend': (3, 7)       # (x, y) proporção média de com recebimento (a cada x, y é recebido)
        }
    }
}

In [4]:
# parâmetros transacionais
pr_trans = {
    'protocolo': {
        'min' : 10000,                         # quantidade mínima de protocolos por entidade entidade
        'max': 30000,                          # quantidade máxima de protocolos por entidade entidade
        'data_inicial': (2020, 1, 1),          # data inicial da massa de dados
        'dias_ant_ult_task': {                 # qtd de dias antes de hoje
            'min': 3,
            'max': 10
        },
        'dias_av_apos_ult_task': {             # qtd de dias de avaliação após última task
            'min': 3,
            'max': 10
        },
        'ult_dia': 3,                          # qtd de dias antes de hoje
        'prop_rating': (15, 25, 20, 30, 10),   # proporção de notas (1, 2, 3, 4, 5)
        'prop_tem_rating': (7, 3),             # proporção média de ações com ratings - (com rating, sem rating)
        'prop_tem_motivo_rating': (7, 3)       # proporção de que tem motivo de rating - (com motivo, sem motivo)
    }
}

# <font color='black'>lê arquivos externos</font>

In [5]:
# abrindo os arquivos...
home = expanduser("~")
caminho_in = './dados_externos/'
caminho_out = home + '/Downloads/GD/'

arq_form = caminho_in + "modelo_form.csv"
arq_municipios = caminho_in + "municipios.csv"
arq_estados = caminho_in + "estados.csv"

df_form = pd.read_csv(arq_form, sep = ',', dtype = str)
df_municipios = pd.read_csv(arq_municipios, sep = ',', dtype = str)
df_estados = pd.read_csv(arq_estados, sep = ',', dtype = str)


print("forms - linhas = %s , colunas = %s " % df_form.shape)
print("municipios - linhas = %s , colunas = %s " % df_municipios.shape)
print("estados - linhas = %s , colunas = %s " % df_estados.shape)

forms - linhas = 621 , colunas = 6 
municipios - linhas = 5570 , colunas = 6 
estados - linhas = 27 , colunas = 5 


In [6]:
# gera uma lista de (cidades, estado)
cols = ['nome_x', 'uf']
df_cidades = pd.merge(df_municipios, df_estados, on='codigo_uf')[cols]
lst_municipios = df_cidades.values.tolist()

# gera uma lista de atributos de form
cols_sel = ['nome', 'tipo']
lst_form = df_form[cols_sel].drop_duplicates(subset = cols_sel).values.tolist()
# lst_form

# <font color='black'>gera Cidades e Estados para Formulários e Entidades</font>

In [7]:
# estipula a quantidade de cidades fará parte do dataset
qtd_cidades = rd.randint(pr['cidades'][0],pr['cidades'][1])

# gera a lista de cidades
lst_cidades = rd.sample(lst_municipios, k = qtd_cidades)

print('cidades =', len(lst_cidades))
# lst_cidades

cidades = 28


# <font color='black'>gera Entidades</font>

In [8]:
# estipula a quantidade de entidades fará parte do dataset
qtd_entidades = int(pr['entidades'] * len(lst_cidades))

# gera a lista de cidades
lst_tmp_entidades = rd.sample(lst_municipios, k = qtd_entidades)

# define orgãos de governo
lst_tmp_orgao = ['Prefeitura de ', 'Secretaria de ']
lst_entidades = []

# gera a lista de entidades
for i in lst_tmp_entidades:
    lst_entidades.append(
        [
            rd.choices(lst_tmp_orgao, [4, 1], k =1)[0] + i[0],
            i[1]
        ]
    )
    
print('entidades =', len(lst_entidades))
# lst_entidades

entidades = 22


# <font color='black'>gera Serviços</font>

## <font color='green'>gera grupos responsável</font>

In [9]:
# gera a lista total de grupos responsáveis que será usada
lst_tmp_grupo_responsavel = []
for i in range(pr['grupo_responsavel']['tot']):
    lst_tmp_grupo_responsavel.append('Grupo Responsável ' + str(i + 1))
print('grupos responsáveis =', len(lst_tmp_grupo_responsavel))
# lst_tmp_grupo_responsavel

grupos responsáveis = 100


## <font color='green'>gera categorias de serviços</font>

In [10]:
# gera a lista total de grupos de serviços que será usada
lst_tmp_categoria_servico = []
for i in range(pr['categoria_servico']['tot']):
    lst_tmp_categoria_servico.append('Categoria de Servico ' + str(i + 1))
print('categorias de serviço =', len(lst_tmp_categoria_servico))
# lst_tmp_categoria_servico

categorias de serviço = 100


## <font color='green'>gera serviços</font>

In [11]:
# gera a lista total de serviços que será usada
lst_tmp_servico = []
for i in range(pr['servico']['tot']):
    lst_tmp_servico.append('Servico ' + str(i + 1))
print('serviços =', len(lst_tmp_servico))
# lst_tmp_servico

serviços = 200


# <font color='black'>gera Usuários</font>

## <font color='green'>gera grupos de usuários</font>

In [12]:
# gera a lista total de grupos de usuários que será usada
lst_tmp_grupo_usuario = []
for i in range(pr['grupo_usuario']['tot']):
    lst_tmp_grupo_usuario.append('Grupo de Usuário ' + str(i + 1))
print('grupos de usuários =', len(lst_tmp_grupo_usuario))
# lst_tmp_grupo_usuario

grupos de usuários = 50


## <font color='green'>gera usuários</font>

In [13]:
# gera a lista total de usuários que será usada
lst_tmp_usuario = []
for i in range(pr['usuario']['tot']):
    lst_tmp_usuario.append('Usuário ' + str(i + 1))
print('usuários =', len(lst_tmp_usuario))
# lst_tmp_usuario

usuários = 100


# <font color='black'>gera Departamentos (encaminhado para)</font>

## <font color='green'>gera departamentos de encaminhamento</font>

In [14]:
# gera a lista total de departamentos que será usada na coluna "encaminhado para"
lst_tmp_departamento = []
for i in range(pr['departamento']['tot']):
    lst_tmp_departamento.append('Departamento ' + str(i + 1))
print('departamentos de encaminhamento =', len(lst_tmp_departamento))
# lst_tmp_departamento

departamentos de encaminhamento = 100


# <font color='black'>Motivos de Cancelamento</font>

## <font color='green'>gera motivos de cancelamento</font>

In [15]:
# gera a lista total de motivos de cancelamento"
lst_tmp_motivos_canc = []
for i in range(pr['motivo_cancelamento']['tot']):
    lst_tmp_motivos_canc.append('Motivo Cancelamento ' + str(i + 1))
print('motivos de encaminhamento =', len(lst_tmp_motivos_canc))
# lst_tmp_motivos_canc

motivos de encaminhamento = 20


# <font color='black'>Motivos de Rating</font>

## <font color='green'>gera motivos de rating</font>

In [16]:
# gera a lista total de motivos de rating"
lst_tmp_motivos_rating = []
for i in range(pr['motivo_rating']['tot']):
    lst_tmp_motivos_rating.append('Motivo de Rating ' + str(i + 1))
print('motivos de rating =', len(lst_tmp_motivos_rating))
# lst_tmp_motivos_rating

motivos de rating = 100


# <font color='black'>gera Ações</font>

## <font color='green'>gera acões</font>

In [17]:
# gera a lista total de ações que será usada
lst_tmp_acao = []
for i in range(pr['acao']['tot']):
    notif = rd.choices((1, 0), pr['acao']['prop_notificacao'])[0]
    lst_tmp_acao.append(('Ação ' + str(i + 1), notif))
print('ações =', len(lst_tmp_acao))
# lst_tmp_acao

ações = 100


## <font color='green'>gera status externo</font>

In [18]:
# gera a lista total de status externo que será usada
lst_tmp_status_ext = []
for i in range(pr['status_ext']['tot']):
    lst_tmp_status_ext.append('Status Externo ' + str(i + 1))
print('status externo =', len(lst_tmp_status_ext))
# lst_tmp_status_ext

status externo = 100


# <font color='black'>Cria estrutura Entidade --> serviços, usuários, departamentos, motivos cancelamento </font>

In [19]:
lst = []

# para cada entidade gera uma coleção de serviços e de usuários
for i in lst_entidades:

##########################################################################################################
# serviços
# estipula a quantidade de serviços será cadastrado para esta entidade
    qtd = rd.randint(pr['servicos_por_entidade']['min'], pr['servicos_por_entidade']['max'])
# define a coleção de serviços para esta entidade
    lst_servicos_da_entidade = rd.sample(lst_tmp_servico, k = qtd)
    
##########################################################################################################
# grupos de responsáveis
# estipula a quantidade de grupos responsáveis será cadastrado para esta entidade
    qtd = rd.randint(pr['grupo_responsavel']['min'], pr['grupo_responsavel']['max'])
# define a coleção de grupos responsáveis para esta entidade
    lst_grupos_responsaveis_da_entidade = rd.sample(lst_tmp_grupo_responsavel, k = qtd)

##########################################################################################################
# categorias de serviços
# estipula a quantidade de categorias de serviços será cadastrado para esta entidade
    qtd = rd.randint(pr['categoria_servico']['min'], pr['categoria_servico']['max'])
# define a coleção de categorias de serviços para esta entidade
    lst_categoria_servicos_da_entidade = rd.sample(lst_tmp_categoria_servico, k = qtd)

##########################################################################################################
# estrutura de serviços - serviço, grupo responsável, categoria de serviço, SLA, faixas SLA, campos do formulário

# escolhe o grupo de responsavel e a categoria de servicos por cada serviço
    lst_serv = []
    for j in lst_servicos_da_entidade:
        # define as faixas de SLA do serviço
        # ((verde_min, verde_máx), (amarelo_min, amarelo_max), (vermelho_min, vermelho_max))
        t2 = rd.randint(pr['servico']['prazo'][0], pr['servico']['prazo'][1])
        t1 = int(t2 * pr['servico']['faixa_amarela'])
        faixas_SLA = ((None, t1), (t1, t2), (t2, None))
        
        # define os atributos de formulário para o serviço
        lst_atributos_form = [
            ['state', 'STRING'],
            ['city', 'STRING'],
            ['neighborhood', 'STRING'],
            ['zipcode', 'STRING'],
            ['street', 'STRING']
        ]
        qtd = rd.randint(pr['servicos_por_entidade']['atributos_form'][0], 
                         pr['servicos_por_entidade']['atributos_form'][1])

        lst_atributos_form.extend(rd.sample(lst_form, k = qtd))
    
        dic_serv = {
            'servico': j,
            'grupo_resp': rd.sample(lst_grupos_responsaveis_da_entidade, k = 1)[0],
            'categoria_serv': rd.sample(lst_categoria_servicos_da_entidade, k = 1)[0],
            'prazo_SLA': t2,
            'faixas_SLA': faixas_SLA,
            'campos_form': lst_atributos_form
        }
        lst_serv.append(dic_serv)

##########################################################################################################
# usuários
# estipula a quantidade de usuários será cadastrado para esta entidade
    qtd = rd.randint(pr['usuários_por_entidade']['min'], pr['usuários_por_entidade']['max'])
# define a coleção de serviços para esta entidade
    lst_usuarios_da_entidade = rd.sample(lst_tmp_usuario, k = qtd)

# estipula a quantidade de grupos de usuários será cadastrado para esta entidade
    qtd = rd.randint(pr['grupos_usuários_por_entidade']['min'], pr['grupos_usuários_por_entidade']['max'])
# define a coleção de grupos de usuários para esta entidade
    lst_grupos_usuario_da_entidade = rd.sample(lst_tmp_grupo_usuario, k = qtd)
    
# escolhe o grupo de usuários por cada usuário
    lst_user = []
    for j in lst_usuarios_da_entidade:
        dic_user = {
            'usuario': j,
            'grupo_usuario': rd.sample(lst_grupos_usuario_da_entidade, k = 1)[0]
        }
        lst_user.append(dic_user)  

##########################################################################################################
# departamentos
# estipula a quantidade de departamentos será cadastrado para esta entidade
    qtd = rd.randint(pr['departamento']['min'], pr['departamento']['max'])
# define a coleção de serviços para esta entidade
    lst_departamentos = rd.sample(lst_tmp_departamento, k = qtd)

##########################################################################################################
# motivos de cancelamento
# estipula a quantidade de motivos de cancelamento será cadastrado para esta entidade
    qtd = rd.randint(pr['motivo_cancelamento']['min'], pr['motivo_cancelamento']['max'])
# define a coleção de motivos de cancelamento para esta entidade
    lst_motivos_canc = rd.sample(lst_tmp_motivos_canc, k = qtd)

##########################################################################################################
# motivos de rating
# estipula a quantidade de motivos de rating será cadastrado para esta entidade
    qtd = rd.randint(pr['motivo_rating']['min'], pr['motivo_rating']['max'])
# define a coleção de motivos de rating para esta entidade
    lst_motivos_rating = rd.sample(lst_tmp_motivos_rating, k = qtd)

##########################################################################################################
# status externo
# estipula a quantidade de motivos de cancelamento será cadastrado para esta entidade
    qtd = rd.randint(pr['status_ext']['min'], pr['status_ext']['max'])
# define a coleção de serviços para esta entidade
    lst_status_ext = rd.sample(lst_tmp_status_ext, k = qtd)

##########################################################################################################
# ações
# estipula a quantidade de ações que será cadastrado para esta entidade
    qtd = rd.randint(pr['acao']['min'], pr['acao']['max'])
# define a coleção de ações para esta entidade
    lst_acoes = rd.sample(lst_tmp_acao, k = qtd)
# define qual será a ação que terá solicitação de apoio
    acao_apoio = rd.sample(lst_acoes, k = 1)[0]
# define qual será a ação que solicitará agendamento
    acao_agenda = rd.sample(lst_acoes, k = 1)[0]

# lista de ações - [ação, 0=não tem apoio ou 1=tem apoio, 0=não tem notificação ou 1=tem notificação]
# refaz a lista de serviço para uma lista de tuplas: (serviço, 0 = não tem apoio ou 1 = tem apoio)
    lst_acoes = [(i[0], 1 if i == acao_apoio else 0, i[1], 1 if i == acao_agenda else 0) for i in lst_acoes]
    
##########################################################################################################
# cidades para os formulários
# estipula a quantidade de cidades serão usadas nos contéudo dos formulários
    qtd = rd.randint(pr['cidades_por_entidade'][0], pr['cidades_por_entidade'][1])
# define a coleção de de cidades serão usadas nos contéudo dos formulários
    lst_cidades_formularios = rd.sample(lst_cidades, k = qtd)
# acrescenta a cidade que originou a entidade
    lst_cidades_formularios.append(i)

# monta estrutura cadastral de 1 entidade para n serviços e n usuários
    dic = {'entidade': i,
           'servicos': lst_serv,
           'usuarios': lst_user,
           'cidades_formularios': lst_cidades_formularios,
           'acoes': lst_acoes,
           'departamentos': lst_departamentos,
           'motivos_canc': lst_motivos_canc,
           'status_externo': lst_status_ext,
           'motivos_rating': lst_motivos_rating
          }
    lst.append(dic)
# lst[0]

# <font color='black'>gera Protocolos</font>

In [20]:
# calcula a quantidade de dias entre a data inicial e hoje
dt_ini = dt.date(pr_trans['protocolo']['data_inicial'][0],
                  pr_trans['protocolo']['data_inicial'][1],
                  pr_trans['protocolo']['data_inicial'][2])
hoje = dt.datetime.now().date()
dias = (hoje - dt_ini).days - pr_trans['protocolo']['ult_dia']

In [21]:
# estipula a data do protocolo
dt_protocolo = dt_ini + dt.timedelta(days = rd.randint(0, dias))

## <font color='green'>gera tasks</font>

In [22]:
# parâmetros da função:
# p = registro do protocolo
# ac = lista de ações a serem soreteadas
# t_min_max = tupla com o tempo mínimo e tempo máximo para sorteio

def Gera_Task(p, deps, ac, tempos, gaps, gap_atend, sta_ext):
    lst_task = []
    cont_task = 0
    tot_task = p['qtd_tasks']
    
##########################################################################################################
# PRIMEIRA AÇÃO - acrescenta uma primeira ação em todos os protocolos
    dic = {'seq_task': 1,
           'acao': ('Abertura de Processo Administrativo', 0, 0, 0),
           'processo_encerrado': 0,
           'processo_cancelado': 0
          }
    lst_task.append(dic)
    cont_task += 1
    
##########################################################################################################
#  AÇÕES INTERMEDIÁRIAS - gera tasks intermediárias (sem a possibilidade de ações iguais consecutivas)

    acao_anterior = None
    for i in range(2, tot_task):
        
        while 1:  # loop para evitar que sorteie acões iguais consecutivas
            ac_e = rd.sample(ac, k = 1)
            if ac_e[0] != acao_anterior:
                break
        acao_anterior = ac_e[0]
        
        dic = {'seq_task': i,
               'acao': ac_e[0],
               'processo_encerrado': 0,
               'processo_cancelado': 0
              }
        lst_task.append(dic)

##########################################################################################################
# ÚLTIMA AÇÃO - se for um protocolo concluído ou cancelado coloca task de conclusão ou cancelamento

    
    dic = {'seq_task': tot_task,
           'acao': (p['situacao']['nome_task'], 0, 0, 0),
           'processo_encerrado': 1 if (p['situacao']['cod'] != 1) else 0,
           'processo_cancelado': 1 if (p['situacao']['cod'] == 3) else 0
          }
    lst_task.append(dic)
    cont_task += 1
    
##########################################################################################################
# COMPLEMENTA A COLEÇÃO DE AÇÕES
    
    # define a data da última task
    total_tasks = len(lst_task)                                             # total de tasks

    # calcula as datas de todas as tasks
    andamento = 1 if p['situacao']['cod'] == 1 else 0                       # flag de task em andamento

    # determina se a task mais recente de um protocolo em andamento tem data de recebimento
    tem_dt_recebimento = rd.choices([0, 1], [gap_atend[2][0], gap_atend[2][1]], k =1)[0]
    
    ult_dt_conclusão = p['dt_ult_task']

    for k in range(total_tasks - 1, -1, -1):

        # sorteia o tempo da task
        tp = rd.uniform(tempos[0], tempos[1])                           
        ult_dt_inicio = ult_dt_conclusão + dt.timedelta(days = -tp)
        
        # sorteia o gap entra uma task e outra task
        gap = rd.uniform(gaps[0], gaps[1])                                      

        # sorteia o gap entre criação da task e o atendimento
        gap_atendimento = rd.uniform(gap_atend[0], gap_atend[1])
        dt_atendimento = ult_dt_inicio + dt.timedelta(days = gap_atendimento)
        
        lst_task[k]['DT_INICIO'] = ult_dt_inicio
        lst_task[k]['TEMPO_ACAO'] = tp
        
        # determina o status externo da ação
        if lst_task[k]['seq_task'] == 1:
            lst_task[k]['status_ext'] = 'Solicitação recebida'
        elif lst_task[k]['seq_task'] == total_tasks:
            if p['situacao']['cod'] == 1:
                lst_task[k]['status_ext'] = rd.sample(sta_ext, k = 1)[0]
            else:
                lst_task[k]['status_ext'] = 'Solicitação Finalizada'
        else:
            lst_task[k]['status_ext'] = rd.sample(sta_ext, k = 1)[0]
        
        # determina data de atendimento e data de fim da ação
        if (lst_task[k]['seq_task'] == total_tasks and andamento == 1):
            
            lst_task[k]['DT_ATENDIMENTO'] = dt_atendimento if tem_dt_recebimento == 1 else ''
            lst_task[k]['DT_FIM'] = ''

        else:
            lst_task[k]['DT_ATENDIMENTO'] = dt_atendimento
            lst_task[k]['DT_FIM'] = ult_dt_conclusão
                
        lst_task[k]['GAP'] = gap    
        lst_task[k]['ENCAMINHADO'] = gap
        lst_task[k]['ENCAMINHADO'] = rd.sample(deps, k = 1)[0]
        
        ult_dt_conclusão = ult_dt_inicio + dt.timedelta(days = -gap)
        
    return lst_task

## <font color='green'>gera formulário</font>

In [23]:
# gera o conteúdo dos formulários para cada protocolo criado
# recebe como parametro o serviço

def Gera_Form(campos_form, cidades, interv_integer, interv_string):
    lst_conteudo = copy.deepcopy(campos_form)
# escolhe qual será a cidade e estado do formulário deste protocolo
    cidade_estado_form = rd.sample(cidades, k = 1)[0]
    
    for i in lst_conteudo:
# testa se o campo do formulário é cidade ou UF para pegar o escolhido acima
        if i[0] == 'city':
            i.append(cidade_estado_form[0])
        elif i[0] == 'state':
            i.append(cidade_estado_form[1])
            
# se não for cidade ou UF faz a escolha aleatória
        else: 
            if i[1] == 'BOOLEAN':
                i.append(rd.randint(0, 1))
            elif i[1] == 'INTEGER':
                i.append(rd.randint(interv_integer[0], interv_integer[1]))
            else:
                cc = 'Valor Campo ' + i[0] + ' ' + str(rd.randint(interv_string[0], interv_string[1]))
                i.append(cc)
            
    return lst_conteudo

## <font color='green'>gera protocolos para cada entidade</font>

In [24]:
lstent_all_protocolo = []
hoje = dt.datetime.now()

# para cada entidade gera uma coleção de serviços e de usuários
for idx, i in enumerate(lst):
    
# protocolos
    lstent_protocolo = []
# estipula a quantidade de protocolos será simulado para esta entidade
    qtd = rd.randint(pr_trans['protocolo']['min'], pr_trans['protocolo']['max'])

    # define a coleção de protocolos para esta entidade
    for k in range(qtd):
        # define o código do protocolo
        codigo_protocolo = str(idx + 1) + '.pr-' + str(k + 1)
        
        # determina por sorteio o status do protocolo ("Concluído", "Em Andamento", "Cancelado") 
        ret_status = (pr['tasks_por_protocolo']['concluido'],
                      pr['tasks_por_protocolo']['andamento'],
                      pr['tasks_por_protocolo']['cancelado'])
        ret_peso = (pr['tasks_por_protocolo']['concluido']['peso_prop'],
                    pr['tasks_por_protocolo']['andamento']['peso_prop'],
                    pr['tasks_por_protocolo']['cancelado']['peso_prop'])
        sit = rd.choices(ret_status, ret_peso, k = 1)[0]
                
        # determina o serviço, a categoria de serviços e o grupo responsável do protocolo
        serv_protocolo = rd.sample(i['servicos'], k = 1)[0]
        formulario = Gera_Form(serv_protocolo['campos_form'],
                               i['cidades_formularios'],
                               pr['servicos_por_entidade']['atributo_integer'],
                               pr['servicos_por_entidade']['atributo_string'])

        # determina o usuário e grupo de usuário do protocolo
        user_protocolo = rd.sample(i['usuarios'], k = 1)[0]        
        
        # determina por sorteio a quantidade de tasks do protocolo
        qtd_tasks = rd.randint(sit['min'], sit['max'])
        
        # determina o motivo do cancelamento
        motivo_canc = rd.sample(i['motivos_canc'], k = 1)[0] if sit['cod'] == 3 else ''
        
        # determina a data e hora da última task e a data da avaliação                
        dias = rd.uniform(pr_trans['protocolo']['dias_ant_ult_task']['min'],    # total de dias para tirar de hoje
                          pr_trans['protocolo']['dias_ant_ult_task']['max'])
        dt_ult_task = hoje + dt.timedelta(days = -dias)                         # data de conclusão da última task

##########################################################################################################
# rating
# determina o rating para protocolos com status "concluído e se terá rating
        n_rating = None; motivo_rating = None; dt_rating = None
        if (sit['cod'] == 2) and (rd.choices((1, 0), pr_trans['protocolo']['prop_tem_rating'], k = 1)[0] == 1):
            n_rating = rd.choices((1, 2, 3, 4, 5), pr_trans['protocolo']['prop_rating'], k = 1)[0]

            # sorteia se vai ter ou não motivo de avaliação
            tem_motivo_rating = rd.choices((1, 0), pr_trans['protocolo']['prop_tem_motivo_rating'], k = 1)[0]
            if tem_motivo_rating == 1:
                motivo_rating = rd.sample(i['motivos_rating'], k = 1)[0]

            # calcula a data de avaliação
            dias = rd.uniform(pr_trans['protocolo']['dias_av_apos_ult_task']['min'],
                              pr_trans['protocolo']['dias_av_apos_ult_task']['max'])
            dt_rating = hoje + dt.timedelta(days = -dias)

        rating = [n_rating, motivo_rating, dt_rating]
        
        # monta a estrutura da task
        dic = {'cod_protocolo': codigo_protocolo,
               'situacao': sit,
               'qtd_tasks': qtd_tasks,
               'dt_ult_task': dt_ult_task,
               'servico': serv_protocolo,
               'usuario': user_protocolo,
               'motivo_canc': motivo_canc,
               'rating': rating,
               'formulario': formulario
              }
        
        # tasks
        dic['tasks'] = Gera_Task(
                                 dic,
                                 i['departamentos'],
                                 i['acoes'], 
                                 (pr['tasks_por_protocolo']['tempos']['min'], 
                                  pr['tasks_por_protocolo']['tempos']['max']),
                                 (pr['tasks_por_protocolo']['gap_entre_tasks']['min'], 
                                  pr['tasks_por_protocolo']['gap_entre_tasks']['max']),
                                 (pr['tasks_por_protocolo']['gap_atendimento']['min'], 
                                  pr['tasks_por_protocolo']['gap_atendimento']['max'],
                                  pr['tasks_por_protocolo']['gap_atendimento']['prop_anda_atend']),
                                  i['status_externo']
                        )
        
        lstent_all_protocolo.append(dic)
        lstent_protocolo.append(dic)
        
    lst[idx]['protocolos'] = lstent_protocolo
# lstent_all_protocolo

# <font color='black'>gera Arquivo Externo</font>

In [25]:
def RetCodigoEntidade(entidade):
    return 'GD_' + entidade.replace(' ', '_')

In [26]:
def RetDt(d):
    if d == '':
        return ''
    else:
        return d.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + 'Z'

## <font color='green'>gera dataset para tasks.csv</font>

In [27]:
lst_tasks = []
for i in lst:
    for k in i['protocolos']:
        for m in k['tasks']:
            lst_tasks.append(
                 [i['entidade'][0],                               # Entidade
                 RetCodigoEntidade(i['entidade'][0]),             # Entidade - código
                 k['servico']['servico'],                         # Serviço
                 k['cod_protocolo'],                              # Protocolo
                 k['usuario']['usuario'],                         # Usuário
                 k['usuario']['grupo_usuario'],                   # Grupo
                 RetDt(m['DT_FIM']),                              # Data e Hora da conclusão
                 RetDt(m['DT_ATENDIMENTO']),                      # Data e Hora de atendimento
                 RetDt(m['DT_INICIO']),                           # Data e Hora de criação
                 m['acao'][0],                                    # Ação
                 m['ENCAMINHADO'],                                # Encaminhado para
                 '',                                              # Comentário
                 m['acao'][1],                                    # Apoio
                 m['processo_encerrado'],                         # Processo encerrado
                 m['processo_cancelado'],                         # Processo cancelado
                 k['motivo_canc'],                                # Motivo de cancelamento
                 m['acao'][2],                                    # Notificação
                 m['status_ext'],                                 # Status externo
                 m['acao'][3],                                    # Agendamento
                 '',                                              # Data de agendamento
                 k['servico']['categoria_serv'],                  # Categoria
                 k['servico']['grupo_resp'],                      # Grupo responsável
                 k['servico']['prazo_SLA']                        # Prazo (em segundos)
                ]                                
            )
print('tasks =', len(lst_tasks))
# lst_tasks

tasks = 3102441


## <font color='green'>gera dataset para sla.csv</font>

In [28]:
tup_faixas = ('Dentro do prazo', 'Perto do prazo', 'Fora do prazo')
lst_sla = []
for i in lst:
    for s in i['servicos']:
        for f in range(3):
            
            lst_sla.append(
                [
                    RetCodigoEntidade(i['entidade'][0]),              # Entidade - código
                    s['servico'],
                    tup_faixas[f],
                    'null' if s['faixas_SLA'][f][0] == None else s['faixas_SLA'][f][0],
                    'null' if s['faixas_SLA'][f][1] == None else s['faixas_SLA'][f][1],
                ]
            )
print('sla =', len(lst_sla))
# lst_sla

sla = 1059


## <font color='green'>gera dataset para rating.csv</font>

In [29]:
lst_rating = []
for i in lst:
    for p in i['protocolos']:
        if p['rating'][0] != None:
            lst_rating.append(
                [
                    p['cod_protocolo'],
                    p['rating'][0],
                    '' if p['rating'][1] == None else p['rating'][1],
                    '' if p['rating'][2] == None else RetDt(p['rating'][2])
                ]
            )
print('rating =', len(lst_rating))
# lst_rating

rating = 199547


## <font color='green'>gera dataset para form.csv</font>

In [30]:
lst_form = []
for i in lst:
    for p in i['protocolos']:
        for f in p['formulario']:

            lst_form.append(
                [
                    f[0].lower().replace(" ", ""),
                    f[0],
                    f[2],
                    p['cod_protocolo'],
                    p['servico']['servico'],
                    f[1]
                ]
            )
print('form =', len(lst_form))
# lst_form

form = 5312784


# <font color='black'>Grava Arquivo Externo CSV</font>

## <font color='green'>gera arquivo tasks.csv</font>

In [31]:
colunas = [['Entidade',
           'Entidade - código',
           'Serviço',
           'Protocolo',
           'Usuário',
           'Grupo',
           'Data e Hora de conclusão',
           'Data e Hora de atendimento',
           'Data e Hora de criação',
           'Ação',
           'Encaminhado para',
           'Comentário',
           'Apoio',
           'Processo encerrado',
           'Processo cancelado',
           'Motivo de cancelamento',
           'Notificação',
           'Status externo',
           'Agendamento',
           'Data de agendamento',
           'Categoria',
           'Grupo responsável',
           'Prazo (em segundos)']]

with open(caminho_out + 'n_tasks.csv', 'w',newline = '') as f:
    
    writer = csv.writer(f, delimiter=',', quotechar = '"')
    writer.writerows(colunas)
    writer.writerows(lst_tasks)

## <font color='green'>gera arquivo sla.csv</font>

In [32]:
colunas = [['entityCode',
            'service',
            'status',
            'limiteMinimo',
            'limiteMaximo']]

with open(caminho_out + 'n_sla.csv', 'w',newline = '') as f:
    
    writer = csv.writer(f, delimiter=',', quotechar = '"')
    writer.writerows(colunas)
    writer.writerows(lst_sla)

## <font color='green'>gera arquivo rating.csv</font>

In [33]:
colunas = [['Solicitação',
            'Nota',
            'Motivo',
            'Data da avaliação']]

with open(caminho_out + 'n_rating.csv', 'w',newline = '') as f:
    
    writer = csv.writer(f, delimiter=',', quotechar = '"')
    writer.writerows(colunas)
    writer.writerows(lst_rating)

## <font color='green'>gera arquivo form.csv</font>

In [34]:
colunas = [['atributo',
            'nome',
            'valor',
            'protocolo',
            'servico',
            'tipo']]

with open(caminho_out + 'n_form.csv', 'w',newline = '') as f:
    
    writer = csv.writer(f, delimiter=',', quotechar = '"')
    writer.writerows(colunas)
    writer.writerows(lst_form)

# <font color='black'>Grava Arquivo no MySQL</font>

## <font color='green'>define os scripts das tabelas</font>

In [73]:
script_n_tasks = (
"CREATE TABLE `tb_n_tasks` ("
" `entidade`                     varchar(64) NOT NULL,"
" `cod_entidade`                 varchar(64) NOT NULL,"
" `servico`                      varchar(64) NOT NULL,"
" `protocolo`                    varchar(32) NOT NULL,"
" `usuario`                      varchar(32) DEFAULT NULL,"
" `grupo`                        varchar(32) DEFAULT NULL,"
" `dthr_conclusao`               datetime DEFAULT NULL,"
" `dthr_atendimento`             datetime DEFAULT NULL,"
" `dthr_criacao`                 datetime NOT NULL,"
" `acao`                         varchar(32) DEFAULT NULL,"
" `encaminhado_para`             varchar(32) DEFAULT NULL,"
" `comentario`                   varchar(32) DEFAULT NULL,"
" `apoio`                        smallint DEFAULT NULL,"
" `proc_encerrado`               smallint DEFAULT NULL,"
" `proc_cancelado`               smallint DEFAULT NULL,"
" `motivo_canc`                  varchar(32) DEFAULT NULL,"
" `notificacao`                  smallint DEFAULT NULL,"
" `status_ext`                   varchar(32) DEFAULT NULL,"
" `agendamento`                  smallint DEFAULT NULL,"
" `dthr_agendamento`             datetime DEFAULT NULL,"
" `categoria`                    varchar(32) DEFAULT NULL,"
" `grupo_resp`                   varchar(32) DEFAULT NULL,"
" `prazo`                        int DEFAULT NULL"
")"
)
script_n_sla = (
"CREATE TABLE `tb_n_sla` ("
" `cod_entidade`                 varchar(64) NOT NULL,"
" `servico`                      varchar(64) NOT NULL,"
" `status`                       varchar(32) NOT NULL,"
" `lim_min`                      int NOT NULL,"
" `lim_max`                      int NOT NULL"
")"
)
script_n_rating = (
"CREATE TABLE `tb_n_rating` ("
" `protocolo`                    varchar(32) NOT NULL,"
" `nota`                         smallint NOT NULL,"
" `motivo`                       varchar(32) NOT NULL,"
" `dthr_avaliacao`               datetime NOT NULL"
")"
)
script_n_form = (
"CREATE TABLE `tb_n_form` ("
" `atributo`                     varchar(64) NOT NULL,"
" `nome`                         varchar(64) NOT NULL,"
" `valor`                        varchar(256) DEFAULT NULL,"
" `protocolo`                    varchar(32) NOT NULL,"
" `servico`                      varchar(64) NOT NULL,"
" `tipo`                         varchar(16) NOT NULL"
")"
)

tabelas = {}
tabelas['tb_n_tasks'] = script_n_tasks
tabelas['tb_n_sla'] = script_n_sla
tabelas['tb_n_rating'] = script_n_rating
tabelas['tb_n_form'] = script_n_form

## <font color='green'>cria as tabelas no BD</font>

In [72]:
import mysql.connector
from mysql.connector import Error

My_host = 'localhost'
My_db = 'bd_reco'
My_user = 'gd'
My_pw = 'Alpar@123'

try:
    connection = mysql.connector.connect(host = My_host, database = My_db, user = My_user, password = My_pw)
    cursor = connection.cursor()
    
    for tb in tabelas:
        table_description = tabelas[tb]
        try:
            print("dropando tabela {}: \n".format(tb), end='')
            apaga = xx = 'DROP TABLE IF EXISTS `' + My_db + '`.`' + tb + '`;'
            cursor.execute(apaga)
            print("criando tabela {}: \n".format(tb), end='')
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("já existe.")
            else:
                print(err.msg)
        else:
            print("OK")

    cursor.close()
    connection.close()
    
except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

dropando tabela tb_n_tasks: 
criando tabela tb_n_tasks: 
OK
dropando tabela tb_n_sla: 
criando tabela tb_n_sla: 
OK
dropando tabela tb_n_rating: 
criando tabela tb_n_rating: 
OK
dropando tabela tb_n_form: 
criando tabela tb_n_form: 
OK


# <font color='black'>TESTE</font>